# Arquivo de coleta de dados do BenchMark

## Atualizando o path do sistema

In [ ]:
import os
import sys

current_dir: str = os.path.abspath("../../..")
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)
print(sys.path)


## Importando as bibliotecas internas

In [ ]:
from src.python.libs.benchmark.data import (BenchMetrics,
                                            FIRST_ORDER, SECOND_ORDER, THIRD_ORDER, FOURTH_ORDER, 
                                            FIRST_SIZE, SECOND_SIZE, THIRD_SIZE, FOURTH_SIZE,
                                            BUBBLE_NAME, MERGE_NAME, QUICK_NAME)

from src.python.libs.order_algorithms.bubble_sort import BubbleSort
from src.python.libs.order_algorithms.merge_sort import MergeSort
from src.python.libs.order_algorithms.quick_sort import QuickSort

from src.python.libs.graphics.graphic_generator import GraphicData, GraphicGenerator

## Importando as bibliotecas externas

In [ ]:
from typing import Union
import pandas as pd

## Criando a função para rodar as simulações

In [ ]:
def simulation() -> GraphicData:
    bubble: BubbleSort=  BubbleSort()
    merge: MergeSort = MergeSort()
    quick: QuickSort = QuickSort()
    graphic_data: GraphicData = GraphicData()
    for algorithm in [bubble, merge, quick]:
        for size in [1, 2, 3, 4]:
            for order in [FIRST_ORDER, SECOND_ORDER, THIRD_ORDER, FOURTH_ORDER]:
                metrics: BenchMetrics = BenchMetrics()
                metrics.data_type = order
                algorithm.data_generator(size, order)
                metrics = algorithm.benchmarkSort(metrics)
                print(f"""
                    -------------------------------------------
                    Algoritmo: {metrics.algorithm_name}
                    Ordem: {metrics.data_type}
                    Tamanho: {metrics.list_size}
                    Tempo: {metrics.execution_time}
                    Memoria: {metrics.memory_usage[1] - metrics.memory_usage[0]}
                    Comparações: {metrics.comparations}
                    Trocas: {metrics.swaps}
                    -------------------------------------------
                """)
                graphic_data.insert_data(metrics)
    
    return graphic_data

## Rodando as simulações

In [ ]:
graphic_data: GraphicData = simulation()
print(graphic_data)